In [1]:
import pandas as pd

data = pd.read_csv("../../data/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
print(data.shape)
data.head()

(6425, 9)


,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,validation
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,validation
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,training
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training


In [2]:
length = data.shape[0]
length

6425

In [3]:
import random

# random.seed(33)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 4327 - 0.673
Validation : 1450 - 0.226
Testing : 648 - 0.101


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,testting
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,validation
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,validation


In [5]:
label_count = data['label2'].value_counts()
label_count

non-rumours    4023
true           1067
unverified      697
false           638
Name: label2, dtype: int64

In [6]:
combination = data.apply(lambda row: f"{row['label2']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label2'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,non-rumours,true,unverified,false
Original	0.63	0.17	0.11	0.1
Training	0.63	0.17	0.1	0.1
Validation	0.61	0.17	0.12	0.1
Testting	0.63	0.16	0.11	0.1

Label,non-rumours,true,unverified,false
Original	4023	1067	697	638
Training	2729	722	453	423
Validation	887	244	171	148
Testting	407	101	73	67


In [7]:
topic_count = data['topic'].value_counts()
combination = data.apply(lambda row: f"{row['topic']}_{row['tvt2']}", axis=1).value_counts()
# print(combination)
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

topics = data['topic'].unique().tolist()
topics_simple = [t.split("-")[0] for t in topics]

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 4)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

topics_str = ','.join(topics_simple)
print(f"\nTopic,{topics_str}")
print(f"Original\t{label_ratio(topic_count, topics)}")
for cv, comp in comparison.items():
    print(f"Part-{cv}\t{label_ratio(comp, topics)}")

topics_str = ','.join(topics_simple)
print(f"\nTopic,{topics_str}")
print(f"Original\t{label_raw_value(topic_count, topics)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, topics)}")


Topic,charliehebdo,germanwings,putinmissing,ferguson,ottawashooting,sydneysiege,gurlitt,prince,ebola
Original	0.3236	0.073	0.037	0.1779	0.1385	0.19	0.0215	0.0363	0.0022
Part-training	0.3289	0.0726	0.0351	0.1736	0.1398	0.1932	0.0203	0.0347	0.0018
Part-validation	0.311	0.0731	0.0421	0.191	0.1393	0.1779	0.0234	0.0393	0.0028
Part-testting	0.3164	0.0756	0.0386	0.1775	0.1281	0.196	0.0247	0.0401	0.0031

Topic,charliehebdo,germanwings,putinmissing,ferguson,ottawashooting,sydneysiege,gurlitt,prince,ebola
Original	2079	469	238	1143	890	1221	138	233	14
Training	1423	314	152	751	605	836	88	150	8
Validation	451	106	61	277	202	258	34	57	4
Testting	205	49	25	115	83	127	16	26	2


In [9]:
data.to_csv("../../data/phemernr2_dataset_with_tvt.csv", index=False)